In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 
import re

In [2]:
# sheetn = pd.ExcelFile('C:/Users/Administrator/Desktop/取消订单数据1_2024101619.xlsx')
# sheetn.parse('sheet1')
df = pd.read_excel('C:/Users/Administrator/Desktop/取消订单数据1_2024101619.xlsx', sheet_name='Sheet1')
order_number_list = df['order_number'].to_list()
order_number_list = tuple(order_number_list)
order_number_list

('A2024100109063515',
 'A2024100110070614',
 'A202410011102228',
 'A20241001143714107',
 'A20241001173244105',
 'A2024100119101322',
 'A2024100203581693',
 'A2024100206254620',
 'A2024100210200951',
 'A20241002105759158',
 'A2024100215024115',
 'A2024100216020011',
 'A20241002175442177',
 'A2024100218103923',
 'A20241002183915117',
 'A202410022004337',
 'A2024100220244157',
 'A2024100221221333',
 'A2024100304572368',
 'A2024100308294045',
 'A2024100308575481',
 'A2024100311194471',
 'A20241003115056164',
 'A2024100313075420',
 'A2024100316160155',
 'A2024100321113628',
 'A2024100412285895',
 'A20241004124334151',
 'A2024100420381381',
 'A20241005104231168',
 'A20241005124530145',
 'A20241005145311212',
 'A202410051905437',
 'A2024100609495485',
 'A20241006114512146',
 'A2024100615175982',
 'A2024100707264824',
 'A2024100708402158',
 'A2024100710230889',
 'A2024100713034510',
 'A20241007132927106',
 'A20241007133234118',
 'A2024100809295654',
 'A20241008104803235',
 'A20241008123605193'

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [4]:
sql1 = f'''
SELECT rarg.time, t.order_number, rarg.nsf_rental_v1_3_hundred FROM (SELECT om.order_number, 
replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
 FROM db_digua_business.t_order om
 LEFT JOIN db_digua_business.t_order_risk tor on om.id=tor.order_id) t
LEFT JOIN db_credit.risk_trace_association rta on rta.risk_trace_id=t.trace_id
LEFT JOIN db_credit.risk_antchain_risk_general rarg on rarg.trace_id=rta.trace_id
WHERE t.order_number in {order_number_list};
'''

df_order = query(sql1)
df_order.shape

(97, 3)

In [5]:
sql2 = f'''
SELECT t.order_number, 
replace(case when JSON_VALID(rjsj.rs_decode) THEN JSON_EXTRACT(rjsj.rs_decode, '$.data.resultData.score') end,'"','') as score
 FROM (SELECT om.order_number, 
replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
 FROM db_digua_business.t_order om
 LEFT JOIN db_digua_business.t_order_risk tor on om.id=tor.order_id) t
LEFT JOIN db_credit.risk_trace_association rta on rta.risk_trace_id=t.trace_id
LEFT JOIN db_credit.risk_jixin_score_jx1000212 rjsj on rjsj.trace_id=rta.trace_id
WHERE t.order_number in {order_number_list};
'''
df_order2 = query(sql2)
df_order2.shape


(97, 2)

In [6]:
df_order

,time,order_number,nsf_rental_v1_3_hundred
0,2024-10-01 09:16:10,A2024100109063515,39
1,2024-10-01 10:13:24,A2024100110070614,33
2,2024-10-01 11:06:22,A202410011102228,34
3,2024-10-01 14:52:59,A20241001143714107,33
4,2024-10-01 17:36:10,A20241001173244105,29
...,...,...,...
92,2024-10-16 09:47:23,A2024101609322380,31
93,2024-10-16 10:18:02,A2024101610053729,41
94,2024-10-16 11:16:43,A2024101611025017,27
95,2024-10-16 11:19:32,A2024101611154888,27


In [7]:
df_order2

,order_number,score
0,A2024100109063515,96.04
1,A2024100110070614,44.08
2,A202410011102228,88.86
3,A20241001143714107,8E+1
4,A20241001173244105,83.11
...,...,...
92,A2024101609322380,72.83
93,A2024101610053729,39.94
94,A2024101611025017,33.91
95,A2024101611154888,73.44


In [17]:
df_order_all = pd.merge(df_order, df_order2, on='order_number', how='inner')
df_order_all.rename(columns={'order_number': '订单号', 'score': 'td212'}, inplace=True)
df_order_all['nsf_rental_v1_3_hundred阈值'] = 26
df_order_all['nsf_rental_v1_3_hundred阈值2'] = 27
df_order_all['td212阈值'] = 95
df_order_all['nsf_rental_v1_3_hundred是否命中'] = np.where(df_order_all['nsf_rental_v1_3_hundred'].astype(float)-df_order_all['nsf_rental_v1_3_hundred阈值']>0, 1, 0)
df_order_all['nsf_rental_v1_3_hundred是否命中_new'] = np.where(df_order_all['nsf_rental_v1_3_hundred'].astype(float)-df_order_all['nsf_rental_v1_3_hundred阈值2']>0, 1, 0)
df_order_all['td212是否命中'] = np.where(df_order_all['td212'].astype(float)-df_order_all['td212阈值']>0, 1, 0)
df_order_all['reject'] = np.where((df_order_all['nsf_rental_v1_3_hundred是否命中']==0)&(df_order_all['td212是否命中']==0), 0, 1)
# df_order_all.to_excel('F:/需求/0927强拒1.xlsx')
df_order_all

,time,订单号,nsf_rental_v1_3_hundred,td212,nsf_rental_v1_3_hundred阈值,nsf_rental_v1_3_hundred阈值2,td212阈值,nsf_rental_v1_3_hundred是否命中,nsf_rental_v1_3_hundred是否命中_new,td212是否命中,reject
0,2024-10-01 09:16:10,A2024100109063515,39,96.04,26,27,95,1,1,1,1
1,2024-10-01 10:13:24,A2024100110070614,33,44.08,26,27,95,1,1,0,1
2,2024-10-01 11:06:22,A202410011102228,34,88.86,26,27,95,1,1,0,1
3,2024-10-01 14:52:59,A20241001143714107,33,8E+1,26,27,95,1,1,0,1
4,2024-10-01 17:36:10,A20241001173244105,29,83.11,26,27,95,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
92,2024-10-16 09:47:23,A2024101609322380,31,72.83,26,27,95,1,1,0,1
93,2024-10-16 10:18:02,A2024101610053729,41,39.94,26,27,95,1,1,0,1
94,2024-10-16 11:16:43,A2024101611025017,27,33.91,26,27,95,1,0,0,1
95,2024-10-16 11:19:32,A2024101611154888,27,73.44,26,27,95,1,0,0,1


In [9]:
pass1 = df_order_all[(df_order_all['nsf_rental_v1_3_hundred'].astype(float)<=26)&(df_order_all['td212'].astype(float)<=95)].shape[0]/df_order_all.shape[0]
pass1

0.010309278350515464

In [10]:
reject1 = df_order_all[(df_order_all['nsf_rental_v1_3_hundred'].astype(float)>26)|(df_order_all['td212'].astype(float)>95)].shape[0]/df_order_all.shape[0]
reject1

0.9896907216494846

In [11]:
pass2 = df_order_all[(df_order_all['nsf_rental_v1_3_hundred'].astype(float)<=27)&(df_order_all['td212'].astype(float)<= 95)].shape[0]/df_order_all.shape[0]
pass2

0.12371134020618557

In [12]:
pass2-pass1

0.1134020618556701

In [18]:
df_order_all['时间'] = df_order_all['time'].dt.date
df_order_all.groupby('时间').agg({'订单号': 'count', 'reject': 'sum'})

,订单号,reject
时间,,
2024-10-01,6,6
2024-10-02,11,11
2024-10-03,8,8
2024-10-04,4,4
2024-10-05,4,4
2024-10-06,3,3
2024-10-07,6,5
2024-10-08,7,7
2024-10-09,5,5
